In [ ]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms

In [ ]:
img_Gray = Image.open('C:/Paper_Project/Hangul_Characters_Image128_Grayscale/가/62570_가.png')
img_Gray_array = np.array(img_Gray)

print(img_Gray_array.shape)

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [1]:
import wandb

In [2]:
!wandb login 6473a3a8207ad0ecadea31cd0cbc7e6fb3ab0462

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\gih0109/.netrc


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline


img = cv2.imread('C:/Paper_Project/1.png')



In [ ]:
img.shape

In [ ]:
import torch
import torch.nn as nn

# context와 x의 예시 shape
context_shape = (16, 68, 256)
x_shape = (16, 256, 16, 16)


# 임의의 값으로 채워진 context와 x 텐서 생성
context = torch.rand(context_shape)
x = torch.rand(x_shape)

# 예시로 nn.Linear를 사용하여 context를 변환
# nn.Linear를 사용하여 context 텐서 변환
linear_layer = nn.Linear(context_shape[-1], x_shape[1] * x_shape[2] * x_shape[3])
context_transformed = linear_layer(context)

# 변환된 context를 원하는 크기로 reshape
desired_shape = (x_shape[0], x_shape[1], x_shape[2], x_shape[3])
context_reshaped = context_transformed.view(desired_shape)

In [29]:
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
import torchvision
from torchvision import transforms
import torch.nn.functional as F


import random

In [10]:
cfg = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'vgg19cut': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'N'],
}

class StyleEncoder2(nn.Module):
    def __init__(self, sty_dim=128, cfg=cfg['vgg11']):
        super().__init__()
        # network layers setting
        self.features = self.make_layers(cfg, True)
        self.cont = nn.Linear(512, sty_dim)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        flat = x.view(x.size(0), -1)
        cont = self.cont(flat)

        return cont

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def make_layers(self, cfg, batch_norm=False):
        layers = []
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=False)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=False)]
                in_channels = v
        return nn.Sequential(*layers)


In [56]:
class Korean_StrokeEmbedding:
    def __init__(self,txt_path,classes):
        self.emd_stroke_list = self.read_stroke_txt(txt_path)
        self.classes = classes

    def read_stroke_txt(self, txt_path):
        emd_stroke_list = []
        read_txt = open(txt_path, 'r')
        for emd in read_txt:
            emd_stroke_list.append(emd[:-1])
        return emd_stroke_list

    def embedding(self, indexs):
        stroke_embedding = []
        for index in indexs:
            tmp = []
            unicode_diff = ord(self.classes[index]) - 44032# ord('가')
            for check in self.emd_stroke_list[unicode_diff]:
                tmp.append(int(check))
            stroke_embedding.append(tmp)
        print("stroke list : ")
        print(stroke_embedding)
        return stroke_embedding

class MakeCondition:
    # 모든 contents 이름 바꾸기
    def __init__(self, num_classes, stroke_text_path, style_enc_path, data_classes, language, device):
        self.device = device
        self.dataset_classes = data_classes
        self.num_classes = num_classes
        self.contents_dim = 60 
        self.contents_emb = nn.Embedding(num_classes, self.contents_dim)
        self.korean_stroke_emb = Korean_StrokeEmbedding(txt_path=stroke_text_path,classes=self.dataset_classes)
        # self.style_enc = self.make_style_enc(style_enc_path)
        self.style_enc = StyleEncoder2()
        self.language = language

        self.load_style_weight(style_enc_path)

    # def make_style_enc(self,style_enc_path):
    #     C ,C_in = 32, 1
    #     sty_encoder = style_enc_builder(C_in, C)
    #     checkpoint = torch.load(style_enc_path, map_location=self.device)
    #     tmp_dict = {}
    #     for k, v in checkpoint.items():
    #         if k in sty_encoder.state_dict():
    #             tmp_dict[k] = v
    #     sty_encoder.load_state_dict(tmp_dict)
    #     # frozen sty_encoder
    #     for p in sty_encoder.parameters():
    #         p.requires_grad = False
    #     return sty_encoder.to(self.device)
        
    def load_style_weight(self, style_enc_path):
        checkpoint = torch.load(style_enc_path, map_location=self.device)
        tmp_dict = {}
        for k, v in checkpoint.items():
            if k in self.style_enc.state_dict():
                tmp_dict[k] = v
        self.style_enc.load_state_dict(tmp_dict)
        # frozen sty_enc
        for p in self.style_enc.parameters():
            p.requires_grad = False
        
        self.style_enc.to(self.device)

    def korean_index_to_uni_diff(self, indexs : list):
        char_list = []
        for index in indexs:
            unicode_diff = ord(self.dataset_classes[index]) - 44032# ord('가')
            char_list.append(unicode_diff)
        return char_list

    # def set_charAttr_dim(mode):
    #     pass
    def make_condition(self, images, indexs, mode):
        input_length = images.shape[0]
        # make channel 1 to 3 to input style enc (b, 1, h, w) -> (b, 3, h, w)
        images = images.repeat(1, 3, 1, 1)
        # contents_index = [int(content_index) for content_index in contents_index]
        # style_encoder = style_enc_builder(1,32).to(self.device)
        contents = None
        stroke =  None
        style = None
        style_c = 128
        style_h, style_w = 16, 16
        
        contents_p, stroke_p = random.random(), random.random()
        if mode == 1:
            if contents_p < 0.3:
                contents = torch.zeros(input_length,self.contents_dim)
            else:
                uni_diff_list = torch.LongTensor(self.korean_index_to_uni_diff(indexs))
                print(f"uni diff list: {uni_diff_list}")
                contents = torch.FloatTensor(self.contents_emb(uni_diff_list))

            if stroke_p < 0.3:
                stroke = torch.zeros(input_length,68)
            else:
                stroke =  torch.FloatTensor(self.korean_stroke_emb.embedding(indexs))
            
            if contents_p < 0.3 and stroke_p < 0.3:
                # style = torch.zeros(input_length,style_c, style_h , style_w)
                style = torch.zeros(input_length, style_c)
            else:
                style = self.style_enc(images).cpu()
                # style = style.view(input_length, style_c, -1).cpu()
        elif mode == 2:
            if contents_p < 0.3:
                contents = torch.zeros(input_length,self.contents_dim)
            else:
                contents = torch.FloatTensor(self.korean_index_to_uni_diff(indexs))

            if stroke_p < 0.3:
                stroke = torch.zeros(input_length,68)
            else:
                stroke =  torch.FloatTensor(self.korean_stroke_emb.embedding(indexs))
            
            # style = torch.zeros(input_length,style_c, style_h , style_w)
            style = torch.zeros(input_length, style_c)


        elif mode == 3: #test
            uni_diff_list = torch.LongTensor(self.korean_index_to_uni_diff(indexs))
            contents = torch.FloatTensor(self.contents_emb(uni_diff_list))
            stroke =  torch.FloatTensor(self.korean_stroke_emb.embedding(indexs))
            style = self.style_enc(images).cpu()
            
            # style = style.view(input_length, style_c, -1).cpu()
        condition_dict = {}
        condition_dict["contents"] = contents.to(self.device)
        condition_dict["stroke"] = stroke.to(self.device)
        condition_dict['style'] = style.to(self.device)

        return condition_dict

In [21]:
gpu_num = 0
image_size = 64
input_size = 64
batch_size = 12
num_classes = 11172
lr = 1e-4
n_epochs = 402
use_amp = True
resume_train = True
train_dirs = "H:/data/Hangul_Characters_Image64_radomSampling420_GrayScale"
sample_img_path = f'{train_dirs}/갊/62570_갊.png'
stroke_text_path = "D:/workspace2/KoFont-Diffusion/Ko_diffusion/text_weight/storke_txt.txt"
style_enc_path = "D:/workspace2/KoFont-Diffusion/Ko_diffusion/text_weight/korean_styenc.ckpt"

In [3]:
# Set transform
transforms = torchvision.transforms.Compose([
    # torchvision.transforms.Resize((input_size,input_size)),
    torchvision.transforms.Grayscale(num_output_channels=1),
# # #     torchvision.transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5), (0.5))
])
dataset = torchvision.datasets.ImageFolder(train_dirs,transform=transforms)

In [5]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)

In [15]:
n = range(0,len(dataset),10)
print("len : ",n)
dataset = Subset(dataset, n)

len :  range(0, 99122, 10)


In [57]:
make_condition = MakeCondition(num_classes=num_classes,
                                    stroke_text_path=stroke_text_path,
                                    style_enc_path=style_enc_path,
                                    data_classes=dataset.dataset.classes,
                                    language='korean',
                                    device="cpu"
                                )

In [27]:
img, content = next(iter(dataloader))
print(img.shape)

torch.Size([1, 1, 64, 64])


In [62]:
condition_dict = make_condition.make_condition(images=img, indexs=content, mode=1)

uni diff list: tensor([10886])
stroke list : 
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]


In [59]:
print(condition_dict["contents"].shape)

torch.Size([1, 60])


In [60]:
print(condition_dict["contents"])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [44]:
condition_dict["stroke"].shape

torch.Size([1, 68])

In [45]:
stroke = condition_dict["stroke"]
stroke

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [39]:
stroke = torch.LongTensor(stroke)
stroke

TypeError: expected TensorOptions(dtype=__int64, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)) (got TensorOptions(dtype=float, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)))

In [35]:
emb_test_layer = nn.Embedding(68, 128)

In [46]:
test_list = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]

In [47]:
print(test_list)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [52]:
test_tensor = torch.LongTensor(test_list)
print(test_tensor)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])


In [53]:
result_emb = emb_test_layer(test_tensor)

In [55]:
result_emb.shape

torch.Size([68, 128])